In [ ]:
# First, let's check what's actually being generated
print("🔍 Debugging single track generation:")
print(f"Track name being used: {track_name}")

# Generate one track and examine it
single_track = generator.generate_composition(track_name, length=30)
print(f"Generated track has: {len(single_track.notes)} notes, {len(single_track.rests)} rests")

# Check if the save method exists
if hasattr(generator, 'save_single_track_with_sustaining_instrument'):
    print("✅ Single track save method exists")
else:
    print("❌ Single track save method missing - you need to add it to your class")

# Test with a simple save to see track structure
test_path = Path("/Users/abraxas3d/organ_donor/data/generated/debug_single_track.mid")
generator.save_single_track_with_sustaining_instrument(single_track, test_path, "cello")

# Let's examine the MIDI file structure
import mido
debug_file = mido.MidiFile(test_path)
print(f"\n📊 MIDI file analysis:")
print(f"Number of tracks in file: {len(debug_file.tracks)}")
print(f"MIDI file type: {debug_file.type}")

for i, track in enumerate(debug_file.tracks):
    note_count = sum(1 for msg in track if msg.type == 'note_on' and msg.velocity > 0)
    print(f"Track {i}: {note_count} note_on messages")

In [ ]:
# Create a fresh single-track test file with a unique name
fresh_path = Path("/Users/abraxas3d/organ_donor/data/generated/SINGLE_TRACK_TEST.mid")
generator.save_single_track_with_sustaining_instrument(single_track, fresh_path, "cello")

# Verify it's single track
import mido
test_file = mido.MidiFile(fresh_path)
print(f"✅ Confirmed: {len(test_file.tracks)} track(s) in {fresh_path.name}")

# Close any open GarageBand files first, then open this specific file
import subprocess
subprocess.run(["open", "-a", "GarageBand", str(fresh_path)])

In [ ]:
# Debug why no rests are being generated
print("🔍 Debugging rest generation:")

# Check what's in the content chain (should have both notes and 'rest')
track_name = available_tracks[0]
content_chain = generator.content_chains[track_name]

print(f"Content chain states: {list(content_chain.transitions.keys())}")
print(f"Content chain has 'rest'? {'rest' in content_chain.transitions}")

# Check some transition probabilities
for state, transitions in list(content_chain.transitions.items())[:3]:
    print(f"From '{state}': {transitions}")

# Also check the original track analysis
print(f"\nOriginal analysis showed: {results[track_name]['rests']} rests")

In [ ]:
# Let's manually trace through the MIDI file to see where rests should be detected
import mido

bach_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/bach.mid")

# Analyze track 0 step by step
track = bach_file.tracks[0]
print("🔍 Manual rest detection analysis:")

current_time = 0.0
active_notes = {}  # note -> start_time
last_note_end = 0.0
rest_count = 0

print("First 20 MIDI messages:")
for i, message in enumerate(track):
    current_time += message.time * (500000 / 1000000) / 480  # Convert ticks to seconds roughly
    
    if message.type == 'note_on' and message.velocity > 0:
        # Note starts
        print(f"{i}: Note ON  {message.note} at time {current_time:.3f}")
        
        # Check if there's a gap since last note ended (= rest)
        if current_time > last_note_end and last_note_end > 0:
            rest_duration = current_time - last_note_end
            print(f"   🎵 REST DETECTED: {rest_duration:.3f} seconds")
            rest_count += 1
        
        active_notes[message.note] = current_time
        
    elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
        # Note ends
        if message.note in active_notes:
            start_time = active_notes.pop(message.note)
            duration = current_time - start_time
            print(f"{i}: Note OFF {message.note} at time {current_time:.3f}, duration {duration:.3f}")
            last_note_end = max(last_note_end, current_time)
    
    if i >= 20:  # Just analyze first 20 messages
        break

print(f"\n📊 Manual analysis found {rest_count} rests in first 20 messages")
print(f"Our algorithm found: {results['track_0']['rests']} rests in entire track")

In [ ]:
# Let's see what types of messages are in the MIDI file
import mido

bach_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/bach.mid")
track = bach_file.tracks[0]

print("🔍 All message types in first 30 messages:")
current_time = 0
for i, message in enumerate(track):
    current_time += message.time
    print(f"{i:2d}: {message.type:15} | time:{message.time:4d} | cumulative:{current_time:6d} | {message}")
    
    if i >= 30:
        break

print(f"\n📊 Message type summary:")
message_types = {}
note_on_count = 0
note_off_count = 0

for message in track:
    msg_type = message.type
    message_types[msg_type] = message_types.get(msg_type, 0) + 1
    
    if message.type == 'note_on' and message.velocity > 0:
        note_on_count += 1
    elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
        note_off_count += 1

for msg_type, count in sorted(message_types.items()):
    print(f"  {msg_type}: {count}")

print(f"\nActual note events:")
print(f"  Note ONs: {note_on_count}")
print(f"  Note OFFs: {note_off_count}")

In [ ]:
# Check all tracks in the Bach file
bach_file = mido.MidiFile("/Users/abraxas3d/organ_donor/data/midi_files/songs/bach.mid")
print(f"🎼 Bach file has {len(bach_file.tracks)} tracks total")

for track_num, track in enumerate(bach_file.tracks):
    note_on_count = 0
    note_off_count = 0
    message_count = len(track)
    
    for message in track:
        if message.type == 'note_on' and message.velocity > 0:
            note_on_count += 1
        elif message.type in ['note_off', 'note_on'] and message.velocity == 0:
            note_off_count += 1
    
    print(f"Track {track_num}: {message_count} messages, {note_on_count} note_ons, {note_off_count} note_offs")

# Let's analyze a track that actually has notes
for track_num, track in enumerate(bach_file.tracks):
    if any(msg.type == 'note_on' and msg.velocity > 0 for msg in track):
        print(f"\n🎵 Analyzing Track {track_num} (has notes):")
        
        # Re-run our analysis on a track with actual notes
        extractor = MidiEventExtractor()
        analyzed_track = extractor.extract_track(track, bach_file.ticks_per_beat)
        
        print(f"Found: {len(analyzed_track.notes)} notes, {len(analyzed_track.rests)} rests")
        
        # Show first few notes and their timing
        for i, note in enumerate(analyzed_track.notes[:5]):
            print(f"  Note {i}: pitch {note.pitch}, start {note.start_time:.3f}, duration {note.duration:.3f}")
        
        for i, rest in enumerate(analyzed_track.rests[:5]):
            print(f"  Rest {i}: start {rest.start_time:.3f}, duration {rest.duration:.3f}")
        
        break  # Just analyze the first track with notes

In [ ]:
# Check if ALL notes are perfectly connected
track1_notes = analyzed_track.notes

gaps = []
overlaps = []
perfect_connections = 0

for i in range(len(track1_notes) - 1):
    current_end = track1_notes[i].start_time + track1_notes[i].duration
    next_start = track1_notes[i + 1].start_time
    
    gap = next_start - current_end
    
    if abs(gap) < 0.001:  # Perfectly connected (within 1ms)
        perfect_connections += 1
    elif gap > 0:  # Actual gap (rest)
        gaps.append(gap)
    else:  # Overlap
        overlaps.append(abs(gap))

print(f"📊 Note connection analysis:")
print(f"Perfect connections: {perfect_connections}")
print(f"Actual gaps (rests): {len(gaps)}")
print(f"Overlaps: {len(overlaps)}")

if gaps:
    print(f"Gap sizes: min={min(gaps):.3f}, max={max(gaps):.3f}, avg={sum(gaps)/len(gaps):.3f}")

In [ ]:
# Check for 'rest' states in the Beethoven content chains
print("🔍 Checking Beethoven content chains for rests:")
for track_name, chain in generator.content_chains.items():
    has_rest = 'rest' in chain.transitions
    total_states = len(chain.transitions)
    print(f"{track_name}: has 'rest'? {has_rest}, total states: {total_states}")
    
    if has_rest:
        rest_transitions = chain.transitions['rest']
        print(f"  From 'rest' goes to: {list(rest_transitions.keys())[:5]}...")  # Show first 5

In [ ]:
# Generate composition from Beethoven with rests
available_tracks = list(generator.note_chains.keys())
print(f"Available Beethoven tracks: {available_tracks}")

# Use track_0 (which has lots of rests)
beethoven_track = generator.generate_composition('track_0', length=300)
print(f"Generated Beethoven-style: {len(beethoven_track.notes)} notes, {len(beethoven_track.rests)} rests")

# Save with sustaining instrument
beet_path = Path("/Users/abraxas3d/organ_donor/data/generated/beethoven_with_rests.mid")
generator.save_single_track_with_sustaining_instrument(beethoven_track, beet_path, "cello")

print(f"🎵 Saved Beethoven-inspired composition with natural rests!")
print(f"This should have the breathing patterns of the original!")

# Open in GarageBand  
import subprocess
subprocess.run(["open", "-a", "GarageBand", str(beet_path)])